In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
order_products_train_df = pd.read_csv("../input/order_products__train.csv")
order_products_prior_df = pd.read_csv("../input/order_products__prior.csv")
orders_df = pd.read_csv("../input/orders.csv")
products_df = pd.read_csv("../input/products.csv")
aisles_df = pd.read_csv("../input/aisles.csv")
departments_df = pd.read_csv("../input/departments.csv")

In [ ]:
products_df.head()

In [ ]:
aisles_df.head()

In [ ]:
departments_df.head()

In [ ]:
products_details = pd.merge(left=products_df,right=departments_df,how="left")
products_details = pd.merge(left=products_details,right=aisles_df,how="left")
products_details.head()

Number of products in each department

In [ ]:
plt.figure(figsize=(8,4))
g=sns.countplot(x="department",data=products_details)
g.set_xticklabels(g.get_xticklabels(), rotation=40, ha="right")
plt.show()

**Top 20 aisle** with most number of products

In [ ]:
plt.figure(figsize=(8,4))
top20_aisle=products_details["aisle"].value_counts()[:20].plot(kind="bar",title='Aisles')


In [ ]:

fig,ax = plt.subplots(7,3, figsize=(20,45), gridspec_kw =  dict(hspace=1.4))
for i, j in enumerate(ax.flatten()):
    if(i < len(departments_df["department"])):
        data=products_details[products_details.department == departments_df.loc[i,"department"]].groupby(['aisle']).count()['product_id'].to_frame().reset_index()
        g = sns.barplot(data.aisle, data.product_id ,palette="Blues_d", ax=j)
        j.set_title('Dep: {}'.format(departments_df.loc[i,"department"]))
        j.set(xlabel = "Aisles", ylabel=" Number of products")
        g.set_xticklabels(labels = data.aisle,rotation=90, fontsize=12)


Some points from the above plots:-
1)Ice creams and frozen meals are popular items in frozen department.
2)Bakery department has a good inventory of bread stuffs.
3)Beers are most popular in alcohol category.
4)Asian cusines have high demand.
5)Dairy products have products such as yougurt and cheese in large quantity.

In [ ]:
orders_df.head()

The **order_df** has the data of all the orders and the orders with **eva_set** value as train or test are the last orders of the users.We need to predict for the users in test category that which products would be reordered. 
Lets see 

In [ ]:
order_products_train_df.head()

In [ ]:
order_products_train_df.groupby("add_to_cart_order")["reordered"].aggregate({'reordered_percnt': 'mean'}).sort_values(by="reordered_percnt",ascending= False).reset_index().head(20)

As we can see the most reordered products are the ones which were added to the **cart** early i.e. the most common products(also the most reordered ones) used by customers were added.

**What are the products which are most common buying choice for the customers.**

In [ ]:
order_products_name_train_df = pd.merge(left=order_products_train_df,right=products_df.loc[:,["product_id","product_name"]],on="product_id",how="left")

In [ ]:
common_Products=order_products_name_train_df[order_products_name_train_df.reordered == 1]["product_name"].value_counts().to_frame().reset_index()
plt.figure(figsize=(16,10))
plt.xticks(rotation=90)
sns.barplot(x="product_name", y="index", data=common_Products.head(20))
plt.ylabel('product_name', fontsize=12)
plt.xlabel('count', fontsize=12)
plt.show()

In [ ]:
order_products_name_prior_df = pd.merge(left=order_products_prior_df,right=products_df.loc[:,["product_id","product_name"]],on="product_id",how="left")

In [ ]:
common_Products_prior=order_products_name_prior_df[order_products_name_prior_df.reordered == 1]["product_name"].value_counts().to_frame().reset_index()
plt.figure(figsize=(16,10))
plt.xticks(rotation=90)
sns.barplot(x="product_name", y="index", data=common_Products_prior.head(20))
plt.ylabel('product_name', fontsize=12)
plt.xlabel('count', fontsize=12)
plt.show()

Which are the **departments and aisles** which have most number of sales

In [ ]:
#Calculating only for the last orders(i.e train data) of each customer to get the idea
order_products_name_train_df = pd.merge(left=order_products_name_train_df,right=products_details.loc[:,["product_id","aisle","department"]],on="product_id",how="left")

In [ ]:
common_aisle=order_products_name_train_df["aisle"].value_counts().to_frame().reset_index()
plt.figure(figsize=(16,10))
plt.xticks(rotation=90)
sns.barplot(x="aisle", y="index", data=common_aisle.head(20),palette="Blues_d")
plt.ylabel('aisle', fontsize=12)
plt.xlabel('count', fontsize=12)
plt.show()

In [ ]:
common_aisle=order_products_name_train_df["department"].value_counts().to_frame().reset_index()
plt.figure(figsize=(16,10))
plt.xticks(rotation=90)
sns.barplot(x="department", y="index", data=common_aisle,palette="Blues_d")
plt.ylabel('department', fontsize=12)
plt.xlabel('count', fontsize=12)
plt.show()

Lets calculate the products which were **reordered** in each order from the train data.

In [ ]:
train_group_reordered = order_products_train_df.groupby(["order_id","reordered"])["product_id"].apply(list).reset_index()

In [ ]:
train_group_reordered=train_group_reordered[train_group_reordered.reordered == 1].drop(columns=["reordered"]).reset_index(drop=True)
train_group_reordered.head()

Same process to get the products which are reordered by the users in **prior data**

In [ ]:
order_products_prior_df.head()

In [ ]:
prior_group_reordered = order_products_prior_df.groupby(["order_id","reordered"])["product_id"].apply(list).reset_index()

In [ ]:
prior_group_reordered=prior_group_reordered[prior_group_reordered.reordered == 1].drop(columns=["reordered"])
prior_group_reordered=prior_group_reordered.reset_index(drop=True)
prior_group_reordered.head()

In [ ]:
User_maxOrder=orders_df.groupby("user_id").order_number.aggregate(np.max).sort_values(ascending= False).reset_index()

**order_number** gives us the number of orders placed by a particular user.

In [ ]:

User_maxOrder["order_number"].value_counts()
plt.figure(figsize=(16,10))
g=sns.countplot(x="order_number",data=User_maxOrder)
g.set_xticklabels(g.get_xticklabels(), rotation=40, ha="right")
plt.show()

Users generally make orders with **4 products.**

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(x="order_dow", data=orders_df, color=color[0])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Day of week', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Orders by week day", fontsize=15)
plt.show()

From the above plot :-
0 and 1 would  be weekends (saturday and sunday respectively) as it has the highest frequency of orders.

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(x="order_hour_of_day", data=orders_df, color=color[0])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Hour of day', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Orders by Hour of day", fontsize=15)
plt.show()

**More orders** are placed between the 8th and the 18th hour and it is intutive as well.